### 감정분류 모델 태스크

<br>

[한국인 감정인식을 위한 복합 영상](https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=820)
<br>[감정분류 모델 태스크](https://aifactory.space/task/2674/overvieww)

<br>

In [ ]:
!pip install --upgrade tensorflow==2.10.0

In [1]:
import os
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Teansorflow:", tf.__version__)
print()
print(tf.test.is_built_with_cuda())
print()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print()
print(tf.test.gpu_device_name())
print()
print(device_lib.list_local_devices())

2023-11-15 06:22:27.437113: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 06:22:27.742341: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 06:22:28.608941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-15 06:22:28.609066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: 

Teansorflow: 2.10.0

True

Num GPUs Available:  2

/device:GPU:0

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7589890277320109501
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23542562816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16291562385041011195
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23542562816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2008312571556366386
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6"
xla_global_id: 2144165316
]


2023-11-15 06:22:30.236021: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 06:22:30.383720: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 22451 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2023-11-15 06:22:30.384571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 22451 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6
2023-11-15 06:22:30.386907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 22451 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci 

In [2]:
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
x_train_path = 'dataset/Train'
y_train_path = 'dataset/Train.csv'
#model_save_path = sys.argv[3]

# 데이터 경로 설정
data_dir = x_train_path
batch_size = 16
image_size = (255, 255)
num_classes = 7  # 7가지 감정 클래스

# 데이터 증강 및 전처리
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # horizontal_flip=True,
    # fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
"""
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(255, 255, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# the model so far outputs 3D feature maps (height, width, features)

# 모델 생성
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(255, 255, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # 출력 레이어: 클래스 수에 맞게 설정
    ])
"""

with tf.device('/gpu:0'):
    model = keras.models.Sequential([
        layers.InputLayer(input_shape=(255, 255, 3)),
        keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
        keras.layers.BatchNormalization(), #batch norm layer
        keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
        keras.layers.BatchNormalization(), #batch norm layer
        keras.layers.Conv2D(64, 3, activation='relu', padding='SAME'), #cnn layer
        keras.layers.BatchNormalization(), #batch norm layer

        keras.layers.MaxPooling2D(pool_size=(2, 2)), #pooling layer
        # keras.layers.Dropout(0.5),

        keras.layers.Flatten(),

        keras.layers.Dense(128, activation="relu"),
        keras.layers.BatchNormalization(),
        # keras.layers.Dropout(0.2),

        keras.layers.Dense(num_classes, activation="softmax") # ouput layer
        ])

# 모델 컴파일
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 학습
epochs = 20

model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size
    
)

# 학습된 모델 저장
model_save_path = "model.h5"  # Set the desired path for saving the model
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")

Found 4171 images belonging to 7 classes.
Epoch 1/20
260/260 [==============================] - 19s 72ms/step - loss: 2.1428 - accuracy: 0.2361
Epoch 2/20
260/260 [==============================] - 19s 72ms/step - loss: 1.7341 - accuracy: 0.3511
Epoch 3/20
260/260 [==============================] - 19s 72ms/step - loss: 1.5535 - accuracy: 0.4084
Epoch 4/20
260/260 [==============================] - 19s 72ms/step - loss: 1.4758 - accuracy: 0.4477
Epoch 5/20
260/260 [==============================] - 19s 72ms/step - loss: 1.4659 - accuracy: 0.4505
Epoch 6/20
260/260 [==============================] - 19s 72ms/step - loss: 1.5412 - accuracy: 0.4243
Epoch 7/20
260/260 [==============================] - 19s 72ms/step - loss: 1.5893 - accuracy: 0.3945
Epoch 8/20
260/260 [==============================] - 19s 72ms/step - loss: 1.2863 - accuracy: 0.5237
Epoch 9/20
260/260 [==============================] - 19s 72ms/step - loss: 1.2905 - accuracy: 0.5155
Epoch 10/20
260/260 [===================

In [14]:
import os
import cv2
import sys
import pandas as pd
import numpy as np
import math
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence

In [15]:
model_path = model_save_path
x_test_path = "dataset/Test"
y_pred_save_path = "y_pred.csv"

class Dataloader(Sequence):
    def __init__(self, base_dataset_path, images, batch_size):
        self.base_dataset_path = base_dataset_path
        self.images = images
        self.batch_size = batch_size
        self.indices = np.arange(len(self.images))


    def __len__(self):
        return math.ceil(len(self.images) / self.batch_size)

    def __getitem__(self, idx):
        indices = self.indices[idx * self.batch_size: (idx + 1) * self.batch_size]
        batch_x = [self.images[i] for i in indices]
        batch_images = self.get_imagesets(batch_x)
        batch_images = batch_images.astype('float32') / 255.0
        return batch_images

    def get_imagesets(self, path_list):
        image_list = []
        for image in path_list:
            image_path = os.path.join(self.base_dataset_path, str(image))
            image_list.append(cv2.imread(image_path))
        return np.array(image_list)

def main():
    # 모델을 로드하기
    model = load_model(model_path)

    inference_dataset = Dataloader(x_test_path, os.listdir(x_test_path), 64)
    pred = model.predict(inference_dataset)
    y_pred_labels = np.argmax(pred, axis=1)

    df = pd.DataFrame(os.listdir(x_test_path), columns=['shuffle_데이터 경로'])
    df['라벨'] = y_pred_labels

    # 모델 추론 결과 저장
    df.to_csv(y_pred_save_path, index=False, encoding='CP949')

if __name__ == '__main__':
    main()

17/17 [==============================] - 2s 116ms/step
